Derived from the notebook found here: https://github.com/Sachin-Wani/deeplearning.ai-GANs-Specialization/blob/master/Course%201%20-%20Build%20Basic%20Generative%20Adversarial%20Networks%20(GANs)/Week%201/C1W1_Your_First_GAN.ipynb

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.nn import (
    BatchNorm1d, BCEWithLogitsLoss, Identity, LeakyReLU, Linear, Module, 
    ReLU, Sequential, Sigmoid)
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm

In [2]:
torch.manual_seed(0)

In [3]:
def plot_tensor_imgs(img_tensor, n_img=25, size=(1, 28, 28)):
    img = img_tensor.detach().cpu().view(-1, *size)
    img_grid = make_grid(img[:n_img], nrow=5)
    plt.imshow(img_grid.permute(1, 2, 0).squeeze())

### Generator

In [4]:
def get_generator_block(in_dim, out_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dims.
    Args:
        in_dim (int): dim of the input vector
        out_dim (int): dim of the output vector
    Returns:
        a generator neural network layer
    '''
    
    return Sequential(Linear(in_dim, out_dim),
                      BatchNorm1d(out_dim),
                      ReLU(inplace=True))

In [5]:
def test_gen_block(in_features, out_features, num_test=1000):
    block = get_generator_block(in_features, out_features)
    assert len(block) == 3, 'block has wrong len'
    assert type(block[0]) == Linear, 'block[0] not Linear'
    assert type(block[1]) == BatchNorm1d, 'block[1] not BatchNorm'
    assert type(block[2]) == ReLU, 'block[2] not ReLU'
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features), \
        (f'output_shape = {tuple(test_output.shape)}, '
         f'expected {(n_test, out_features)}')
    assert test_output.std() > 0.55, 'output sd <= 0.55'
    assert test_output.std() < 0.65, 'output sd >= 0.65'

test_gen_block(25, 12)
test_gen_block(15, 28)
print("Success!")

Success!


In [6]:
class Generator(Module):
    '''
    Generator Class
    Vals:
        z_dim (int): dim of the noise vector
        img_dim: the dimension of the images, fitted for the dataset used
          (Defaults to MNIST images: 28 x 28 = 784)
        hidden_dim (int): dime of inner layer
    '''
    def __init__(self, z_dim=10, img_dim=28*28, hidden_dim=128):
        super().__init__()
        self.gen = Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, 2*hidden_dim),
            get_generator_block(2*hidden_dim, 4*hidden_dim),
            get_generator_block(4*hidden_dim, 8*hidden_dim),
            Linear(8*hidden_dim, img_dim),
            Sigmoid())
        
    def forward(self, noise):
        '''
        Complete a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Args:
            noise: a noise tensor with dims (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    def get_gen(self):
        return self.gen

In [7]:
def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    
    # Check there are six modules in the sequential part
    assert len(gen) == 6
    assert str(gen.__getitem__(4)).replace(' ', '') \
        == (f'Linear(in_features={hidden_dim * 8},'
            f'out_features={im_dim},bias=True)')
    assert str(gen.__getitem__(5)).replace(' ', '') == 'Sigmoid()'
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)

    # Check that the output shape is correct
    assert tuple(test_output.shape) == (num_test, im_dim)
    assert test_output.max() < 1, "Make sure to use a sigmoid"
    assert test_output.min() > 0, "Make sure to use a sigmoid"
    assert test_output.std() > 0.05, "Don't use batchnorm here"
    assert test_output.std() < 0.15, "Don't use batchnorm here"

test_generator(5, 10, 20)
test_generator(20, 8, 24)
print("Success!")

Success!


### Noise

In [8]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions 
    (n_samples, z_dim), creates a tensor of that shape filled with random 
    numbers from the normal distribution.
    Args:
      n_samples (int): the number of samples to generate, a scalar
      z_dim (int): the dimension of the noise vector, a scalar
      device (str): the device type ['cpu' | 'gpu']
    '''
    return torch.randn(size=(n_samples, z_dim), device=device)

In [9]:
def test_get_noise(n_samples, z_dim, device='cpu'):
    noise = get_noise(n_samples, z_dim, device)
    assert tuple(noise.shape) == (n_samples, z_dim)
    assert torch.abs(noise.std() - torch.tensor(1.0)) < 0.01
    assert str(noise.device) == device

test_get_noise(1000, 100, 'cpu')
try:
    test_get_noise(1000, 32, 'cuda')
except Exception as e: 
    print(e)
print("Success!")

Torch not compiled with CUDA enabled
Success!


### Discriminator

In [10]:
def get_discriminator_block(in_dim, out_dim, neg_slope=0.2):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given 
    input and output dimensions.
    Args:
        in_dim (int): the dimension of the input vector
        out_dim (int): the dimension of the output vector
    Returns:
        A discriminator neural network layer, with a linear transformation 
        followed by an nn.LeakyReLU activation with negative slope of 0.2 
        https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html
    '''
    return Sequential(Linear(in_dim, out_dim),
                      LeakyReLU(neg_slope))

In [11]:
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)

    # Check there are two parts
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)

    # Check that the shape is right
    assert tuple(test_output.shape) == (num_test, out_features)
    
    # Check that the LeakyReLU slope is about 0.2
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5

test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


In [12]:
class Discriminator(Module):
    '''
    Discriminator Class
    Values:
        im_dim (int): the dimension of the images, fitted for the dataset 
        used
            (MNIST images are 28x28 = 784 so that is the default)
        hidden_dim (int): n nodes in the inner layer
    '''
    def __init__(self, im_dim=784, hidden_dim=128):
        super().__init__()
        self.disc = Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            Linear(hidden_dim, 1))

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: 
        Given an image tensor, returns a 1-dimension tensor representing
          fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image)
    
    # Needed for grading
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [13]:
def test_discriminator(z_dim, hidden_dim, num_test=100):
    
    disc = Discriminator(z_dim, hidden_dim).get_disc()

    # Check there are three parts
    assert len(disc) == 4
    assert type(disc.__getitem__(3)) == Linear

    # Check the linear layer is correct
    test_input = torch.randn(num_test, z_dim)
    test_output = disc(test_input)
    assert tuple(test_output.shape) == (num_test, 1)

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

Success!


## Training

In [14]:
# Set your parameters
criterion = BCEWithLogitsLoss()
n_epochs = 200
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001
device = 'cpu' #'cuda'
# Load MNIST dataset as tensors
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [15]:
gen = Generator(z_dim).to(device)
gen_opt = Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device)
disc_opt = Adam(disc.parameters(), lr=lr)

In [16]:
for real, _ in dataloader:
    print(type(real))
    break

<class 'torch.Tensor'>


In [17]:
classes = ['fake'] * 4 + ['real'] * 4
labels = [0, 0, 0, 0, 1, 1, 1, 1]
n = len(labels)
inds = list(range(n))
np.random.shuffle(inds)

classes = [classes[i] for i in inds]
labels = [labels[i] for i in inds]

for c, l in zip(classes, labels):
    print(f'{c}: {l}')

fake: 0
real: 1
fake: 0
fake: 0
real: 1
real: 1
real: 1
fake: 0


In [18]:
def shuffle(classes, labels):
    n = len(classes)
    inds = np.random.shuffle(list(range(n)))
    classes = [classes[i] for i in inds]
    labels = [labels[i] for i in inds]
    return classes, labels

In [19]:
def get_disc_loss(
        gen, disc, criterion, real, n_images, z_dim, device='cpu'):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given 
          z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional 
          prediction of real/fake
        criterion: the loss function, which should be used to compare 
          the discriminator's predictions to the ground truth reality of
          the images (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
          which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    # These are the steps you will need to complete:
    # 1) Create noise vectors and generate a batch (n_images) of fake 
    #    images. Make sure to pass the device argument to the noise.
    # 2) Get the discriminator's prediction of the fake image and 
    #    calculate the loss. Don't forget to detach the generator!
    #    (Remember the loss function you set earlier -- criterion. You 
    #     need a 'ground truth' tensor in order to calculate the loss. 
    #     For example, a ground truth tensor for a fake image is all 
    #     zeros.)
    # 3) Get the discriminator's prediction of the real image and
    #    calculate the loss.
    # 4) Calculate the discriminator's loss by averaging the real and fake 
    #    loss and set it to disc_loss.
    #    *Important*: You should NOT write your own loss function here -
    #    use criterion(pred, true)!
    #### START CODE HERE ####
    noise_samples = get_noise(n_images, z_dim, device=device)
    fakes = [gen(noise) for noise in noise_samples]
    #inputs = real + fakes
    #labels = [1]*len(real) + [0]*len(fakes)
    #inputs, labels = shuffle(inputs, labels)
    #preds = [disc(img) for img in inputs]
    #losses = [criterion(pred, actual) 
    #          for pred, actual in zip(preds, labels)]
    #disc_loss = np.array(losses).mean()
    disc_loss = []
    for fake in fakes:
        pred = disc(fake)
        loss = criterion(pred, 0)
        print('loss:', loss)
        disc_loss.append(loss)
    for img in real:
        pred = disc(img)
        loss = criterion(pred, 1)
        disc_loss.append(loss)
    disc_loss = np.array(disc_loss).mean()
    #### END CODE HERE ####
    return disc_loss

In [20]:
def test_disc_reasonable(num_images=10):    
    gen = torch.zeros_like
    disc = Identity()
    criterion = torch.mul # Multiply
    real = torch.ones(num_images, 1)
    assert torch.all(
        torch.abs(
            get_disc_loss(
                gen, disc, criterion, real, num_images, z_dim, 'cpu'
            ) - 0.5
        ) < 1e-5)

    gen = torch.ones_like
    disc = Identity()
    criterion = torch.mul # Multiply
    real = torch.zeros(num_images, 1)
    assert torch.all(
        torch.abs(
            get_disc_loss(
                gen, disc, criterion, real, num_images, z_dim, 'cpu')
        ) < 1e-5)

In [21]:
test_disc_reasonable()

loss: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
loss: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
loss: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
loss: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0

<ipython-input-19-6a8a3dad07ae>:56: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  disc_loss = np.array(disc_loss).mean()
<ipython-input-19-6a8a3dad07ae>:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  disc_loss = np.array(disc_loss).mean()


AttributeError: 'torch.dtype' object has no attribute 'type'